In [7]:
import torch
import numpy as np
import pandas as pd

In [23]:
mask = torch.load('test_edge_index.pt').transpose_(1,0).numpy()

In [24]:
df = pd.read_csv('data/adult_nonulls_sample10.csv')
n_rows, n_columns = df.shape

In [31]:
mask_fixed = mask[:len(mask)//2].copy()

In [32]:
mask_fixed[mask_fixed>=n_rows]-=n_rows

In [54]:
df_dirty = df.copy()
for edge in mask_fixed:
    df_dirty.iloc[edge[0], edge[1]] = np.nan

In [58]:
df_dirty.to_csv('data/adult_nonulls_dirty_grape.csv', index=False)

In [87]:
import pickle

results = pickle.load(open('result_grape_unpack_10k_epochs.pkl', 'rb'))

In [88]:
outputs = results['outputs']
label_test = outputs['label_test']
pred_test = outputs['final_pred_test']

In [89]:
sum(label_test==pred_test)/len(label_test)

0.0

In [90]:
df_hivae = pd.read_csv('data/adult_nonulls_sample10_hivae.csv')

In [91]:
from sklearn import preprocessing

In [92]:
scaler = preprocessing.MinMaxScaler()
x = df_hivae.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_scaled = pd.DataFrame(x_scaled)


In [93]:
unique = df_scaled[0].unique()
unique.sort()

In [94]:
col_values = {col: df_scaled[col].unique()  for col in df_scaled.columns}

In [96]:
def closest(val, vector):
    dist = 1
    chosen = None
    for idx, v in enumerate(vector):
        new_dist = np.abs(val - v)
        if new_dist < dist:
            dist = new_dist
            chosen = v
    return chosen

In [112]:
acc_by_col = {k: [] for k in range(len(df.columns))}
for idx, edge in enumerate(mask_fixed):
    row,col = edge
    pred = pred_test[idx]
    label = label_test[idx]

    corrected = []
    label_subset = []

    cc = closest(pred, col_values[col])

    acc_by_col[col].append((cc, label))
    # corrected.append(cc)
    # label_subset.append(label)
    # label_subset = np.array(label_subset)
    # corrected = np.array(corrected)
    # acc = sum(label_subset==corrected)/len(label_subset)
    # acc_by_col[col] = acc

In [126]:
for col in acc_by_col:
    vv = np.array(acc_by_col[col])
    acc = sum(vv[:,0] == vv[:,1])/len(vv)
    print(f'{df.columns[col]:<20}{acc:>10.4f}')

age                     0.0000
workclass               0.3018
education               0.1067
education-num           0.1441
marital-status          0.0636
occupation              0.0109
relationship            0.0000
race                    0.8487
sex                     0.7941
capital-gain            0.3231
capital-loss            0.4565
hours-per-week          0.0011
native-country          0.5590
income                  0.7919
